In [2]:
import ants
import numpy as np
from dipy.tracking.streamline import *
from dipy.io.streamline import *
from scipy.io import loadmat, savemat
import nibabel as nib
import nibabel.streamlines as nis

In [2]:
avgt = nib.load('/Users/mahdi/stage-2022-mahdi/AVGT.nii.gz')
affine = avgt.affine
ref_vol = avgt.get_fdata()
shape = avgt.shape

In [5]:
sft_file = '/Users/mahdi/stage-2022-mahdi/data/allen/allen_wildtype_tractogram.trk'
ref = '/Users/mahdi/stage-2022-mahdi/niftis/allen_template_50_ras.nii.gz'
sft_streamlines, sft_hdr = load_tractogram(filename=sft_file)

In [4]:
ref_allen_ras = nib.load(ref)

In [6]:
save_tractogram(
    fname="/Users/mahdi/stage-2022-mahdi/niftis/tracts/allen_wildtype_tractogram.trk",
    streamlines=sft_streamlines,
    affine=np.eye(4)
    )

In [6]:
print(sft_streamlines[0][0])

[ 4.4000001 -5.5999999 -6.9000001]


In [9]:
def allenPIRmm_allenRASvox(pt):
    B = np.array([[ 0.05,  0,        0    ],
                 [  0,    -0.05,     0    ],
                 [  0,     0,       -0.05 ]])
    vox = np.linalg.inv(B) @ pt

    return vox

In [10]:
print(allenPIRmm_allenRASvox(sft_streamlines[0][0]))

[  88.00000191  111.99999809  138.00000191]


In [20]:
file_t = '/Users/mahdi/stage-2022-mahdi/transformations_allen2avgt/allen2avgtAffine_50.mat'
t = ants.read_transform(file_t)

In [21]:
def allenRASvox_avgtRASvox(pt, t):
    it = t.invert()
    return it.apply_to_point(pt)

In [23]:
streamlines_list = []
for streamline in sft_streamlines:
    strimline = []
    for point in streamline:
        allenRASvox = allenPIRmm_allenRASvox(point)
        avgtRASvox = allenRASvox_avgtRASvox(allenRASvox, t)
        strimline.append(avgtRASvox)
    streamlines_list.append(strimline)

In [24]:
print(len(streamlines_list))

577555


In [30]:
rotation = np.array([
    [1,  0,  0, 0],
    [0, -1,  0, 0],
    [0,  0, -1, 0],
    [0,  0,  0, 1]
    ])

In [31]:
translation = np.array([
    [1, 0, 0,  0  ],
    [0, 1, 0, -212],
    [0, 0, 1, -158],
    [0, 0, 0,  1  ]
    ])

In [32]:
affine_mib_tract = affine @ rotation @ translation

In [33]:
save_tractogram(
    fname="/Users/mahdi/stage-2022-mahdi/niftis/tracts/manually_filled_streamlines.trk",
    streamlines=streamlines_list,
    affine=affine_mib_tract
    )

In [ ]:
save_tractogram(
    fname="/Users/mahdi/stage-2022-mahdi/niftis/tracts/manually_filled_streamlines.tck",
    streamlines=streamlines_list,
    affine=affine_mib_tract
    )